In [ ]:
import os
import warnings
import yaml
import requests
from automation.apis.process_documents import APIClient, PDFHandler

# Suppress warnings
warnings.filterwarnings('ignore')

# Load configuration
try:
    with open("config.yaml", "r") as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Error: config.yaml file not found.")
    exit(1)
except yaml.YAMLError as e:
    print(f"Error reading YAML configuration: {e}")
    exit(1)

# Extract configurations
try:
    user_email = config["usercred"]["user"]
    unprocessed_docs_endpoint = config["apis"]["unprocessed_documents"]
    get_document = config["apis"]["get_documents"]
    insert_key_values = config["apis"]["insert_key_values"]
    insert_step_result = config["apis"]["insert_step_result"]
    create_asset = config["apis"]["create_asset"]
except KeyError as e:
    print(f"Missing key in configuration file: {e}")
    exit(1)

# Initialize API client
client = APIClient()

# Authenticate and retrieve access token
try:
    access_token = client.authenticate(email=user_email)
    if not access_token:
        raise ValueError("Authentication failed. No access token received.")
except Exception as e:
    print(f"Authentication error: {e}")
    exit(1)

# Fetch unprocessed documents
try:
    unprocessed_documents = client.get_document_id(unprocessed_docs_endpoint)
    if not isinstance(unprocessed_documents, list):
        raise ValueError("Unexpected API response format. Expected a list.")
except requests.exceptions.RequestException as e:
    print(f"Error fetching unprocessed documents: {e}")
    exit(1)
except Exception as e:
    print(f"Unexpected error: {e}")
    exit(1)

print("API Response:", unprocessed_documents)

# Process each document
for doc in unprocessed_documents:
    try:
        document_id = doc.get("DocumentId")
        if not document_id:
            print("Warning: Missing DocumentId in response:", doc)
            continue  # Skip to the next document

        # Step 1: Insert Document Key/Value Pairs
        key_values_payload = {
            "DocumentId": document_id,
            "KeyValues": {"Processed": "True"}
        }
        key_values_response = client.make_request(insert_key_values, method="POST", data=key_values_payload)

        if not key_values_response:
            print(f"Warning: Failed to insert key/value pairs for Document ID {document_id}")

        # Step 2: Fetch Document Content
        document_response = client.make_request(f"{get_document}/{document_id}")

        if not document_response:
            print(f"Warning: No response for Document ID {document_id}")
            continue  # Skip processing if the response is empty

        output_file = document_response.get("DocumentName")
        document_content = document_response.get("DocumentContent")

        if output_file and document_content:
            PDFHandler.save_base64_as_pdf(document_content, output_file)
            print(f"Saved document: {output_file}")
        else:
            print(f"Warning: Missing DocumentName or DocumentContent for ID {document_id}")

        # Step 3: Insert Step Result (Success)
        step_result_payload = {
            "DocumentId": document_id,
            "StepName": "DocumentProcessing",
            "ProcessResult": True,
            "Message": f"Document {document_id} processed successfully."
        }
        step_result_response = client.make_request(insert_step_result, method="POST", data=step_result_payload)

        if not step_result_response:
            print(f"Warning: Failed to insert processing step for Document ID {document_id}")

        # Step 4: Create Asset
        create_asset_payload = {
            "DocumentId": document_id,
            "AssetType": "ProcessedDocument",
            "FileName": output_file
        }
        create_asset_response = client.make_request(create_asset, method="POST", data=create_asset_payload)

        if not create_asset_response:
            print(f"Warning: Failed to create asset for Document ID {document_id}")

    except requests.exceptions.RequestException as e:
        print(f"Request error while processing document {document_id}: {e}")
        
        # Step 5: Insert Step Result (Failure)
        step_result_payload = {
            "DocumentId": document_id,
            "StepName": "DocumentProcessing",
            "ProcessResult": False,
            "Message": f"Failed due to request error: {e}"
        }
        client.make_request(insert_step_result, method="POST", data=step_result_payload)
        
    except KeyError as e:
        print(f"Missing expected key in response for Document ID {document_id}: {e}")

        # Step 5: Insert Step Result (Failure)
        step_result_payload = {
            "DocumentId": document_id,
            "StepName": "DocumentProcessing",
            "ProcessResult": False,
            "Message": f"Failed due to missing key: {e}"
        }
        client.make_request(insert_step_result, method="POST", data=step_result_payload)

    except Exception as e:
        print(f"Unexpected error processing Document ID {document_id}: {e}")

        # Step 5: Insert Step Result (Failure)
        step_result_payload = {
            "DocumentId": document_id,
            "StepName": "DocumentProcessing",
            "ProcessResult": False,
            "Message": f"Unexpected error: {e}"
        }
        client.make_request(insert_step_result, method="POST", data=step_result_payload)

print("Processing complete.")
